In [16]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from osgeo import gdal
from rasterio import rio

In [2]:
import pandas as pd

In [3]:
from datetime import date

In [4]:
import requests

In [5]:
import threading
import multiprocessing

In [6]:
import os
import shutil
from glob import glob
import zipfile

In [7]:
USERNAME = ["barmasushma", "barmasushma1", "SumanjaliDamarla"]
PASSWORD = ["barmasushma", "barmasushma1", "D.Sumanjali1"]
API_SOURCE = "https://scihub.copernicus.eu/dhus"
PROCESSING_LEVEL = ["Level-1", "Level-2"]
SATELLITE = "Sentinel-2"
STUDY_AREA = "StudyArea.geojson"

In [8]:
footprint = geojson_to_wkt(read_geojson(STUDY_AREA))

In [9]:
api = []
for i in range(len(USERNAME)):
    api.append(SentinelAPI(USERNAME[i], PASSWORD[i], API_SOURCE))

In [10]:
DAYS = {
    "jan" : '31',
    "feb" : '28',
    "mar" : '31',
    "apr" : '30',
    "may" : '31',
    "jun" : '30',
    "jul" : '31',
    "aug" : '31',
    "sep" : '30',
    "oct" : '31',
    "nov" : '30',
    "dec" : '31'
}
MONTHS = {
    "jan" : '01',
    "feb" : '02',
    "mar" : '03',
    "apr" : '04',
    "may" : '05',
    "jun" : '06',
    "jul" : '07',
    "aug" : '08',
    "sep" : '09',
    "oct" : '10',
    "nov" : '11',
    "dec" : '12'
}

In [11]:
NOV_BEGIN = "20181101"
NOV_END = "20181130"

DEC_BEGIN = "20181201"
DEC_END = "20181231"

JAN_BEGIN = "20190101"
JAN_END = "20190131"

FEB_BEGIN = "20190201"
FEB_END = "20190228"

MAR_BEGIN = "20190301"
MAR_END = "20190331"

In [12]:
def queryForProducts(footprint, beg, end, sat):
    return api[0].query(footprint, date=(beg, end),platformname=sat)

In [13]:
def getLevelURLs(df):
    level1 = []
    for i in range(df.shape[0]):
        if PROCESSING_LEVEL[0] in df["processinglevel"][i]:
            level1.append(df["link"][i])

    level2 = []
    for i in range(df.shape[0]):
        if PROCESSING_LEVEL[1] in df["processinglevel"][i]:
            level2.append(df["link"][i])
    
    return level1, level2

In [14]:
def curateLevelProducts(products, df, level):
    keys = list(products.keys())
    for i in range(df.shape[0]):
        if level not in df["processinglevel"][i]:
            products.pop(keys[i])
    return products

In [15]:
def downloadDataUsingThread(prod):
    try:
        t1 = threading.Thread(api[0].download_all(prod))
        t1.start()
        t1.join()
    except AssertionError:
        print("Download of specified products done")
    except:
        print("Something went wrong")

In [ ]:
def createDataDirectory(month):
    data_dir = os.getcwd()+"\\data\\"+month+"_data"
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
    return data_dir
def getDataDirectory(month):
    return os.getcwd()+"\\data\\"+month+"_data"
def createBackupDirectory(month):
    backup = "D:\\"+month+"_data"
    if not os.path.exists(backup):
        os.mkdir(backup)
    return backup
def getBackupDirectory(month):
    return "D:\\"+month+"_data"

In [ ]:
def listDataFiles(data_dir, ext):
    paths = glob(data_dir+"/*"+ext)
    fileNames = []
    for path in paths:
        fileNames.append(path.split('.')[0].split('\\')[-1])
    return paths, fileNames

In [ ]:
def checkAllFiles(data_det, fileNames):
    product_ids = (list(data_det.keys()))
    desired_files = []
    for pid in product_ids:
        desired_files.append(data_det[pid]['title'])
    return sorted(desired_files) == sorted(fileNames)

In [ ]:
def arrangeData(month, curr_path, data_det):
    curr_paths, FileNames = listDataFiles(curr_path, ".zip")
    if not (checkAllFiles(data_det, FileNames)):
        raise Exception("All files are not downloaded")
    else:
        print("All files  Downloaded successfully")
    dst_dir = createDataDirectory(month)
    backup = createBackupDirectory(month)
    for path in curr_paths:
        print("Backup files to "+backup)
        shutil.copy2(path, backup)
        print("Moving files to "+dst_dir)
        shutil.move(path, dst_dir)
    return dst_dir

In [ ]:
def extractFiles(data_dir):
    paths, fileNames = listDataFiles(data_dir, ".zip")
    l = len(paths)
    i = 0
    for path in paths:
        print(str(i)+"/"+str(l)+"  Extracting "+path.split('.')[0].split('\\')[-1]+"...")
        with zipfile.ZipFile(path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)

In [ ]:
def cleanup(data_dir):
    paths, fileNames = listDataFiles(data_dir, ".zip")
    l = len(paths)
    i = 0
    for path in paths:
        print(str(i)+"/"+str(l)+"Deleting "+path.split('.')[0].split('\\')[-1]+"...")
        os.remove(path)

In [ ]:
nov_products = queryForProducts(footprint, NOV_BEGIN, NOV_END,SATELLITE)
dec_products = queryForProducts(footprint, DEC_BEGIN, DEC_END,SATELLITE)
jan_products = queryForProducts(footprint, JAN_BEGIN, JAN_END,SATELLITE)
feb_products = queryForProducts(footprint, FEB_BEGIN, FEB_END,SATELLITE)
mar_products = queryForProducts(footprint, MAR_BEGIN, MAR_END,SATELLITE)

In [ ]:
nov_df = api[0].to_dataframe(nov_products)
dec_df = api[0].to_dataframe(dec_products)
jan_df = api[0].to_dataframe(jan_products)
feb_df = api[0].to_dataframe(feb_products)
mar_df = api[0].to_dataframe(mar_products)

In [ ]:
nov_level1, nov_level2 = getLevelURLs(nov_df)
dec_level1, dec_level2 = getLevelURLs(dec_df)
jan_level1, jan_level2 = getLevelURLs(jan_df)
feb_level1, feb_level2 = getLevelURLs(feb_df)
mar_level1, mar_level2 = getLevelURLs(mar_df)

In [ ]:
nov_data_det = curateLevelProducts(nov_products, nov_df, PROCESSING_LEVEL[1])
dec_data_det = curateLevelProducts(dec_products, dec_df, PROCESSING_LEVEL[1])
jan_data_det = curateLevelProducts(jan_products, jan_df, PROCESSING_LEVEL[1])
feb_data_det = curateLevelProducts(feb_products, feb_df, PROCESSING_LEVEL[1])
mar_data_det = curateLevelProducts(mar_products, mar_df, PROCESSING_LEVEL[1])

In [ ]:
months = [
    "Nov", "Dec", "Jan", "Feb", "Mar"
]
levelCount = [
    [nov_df.shape[0], len(nov_level1), len(nov_level2)],
    [dec_df.shape[0], len(dec_level1), len(dec_level2)],
    [jan_df.shape[0], len(jan_level1), len(jan_level2)],
    [feb_df.shape[0], len(feb_level1), len(feb_level2)],
    [mar_df.shape[0], len(mar_level1), len(mar_level2)]
]

print("      Total  L1  L2")
for i in range(len(months)):
    print(months[i] + "   " + str(levelCount[i][0]) + "     " + str(levelCount[i][1]) + "  " + str(levelCount[i][2]))

In [ ]:
downloadDataUsingThread(nov_data_det)
nov_data_dir = arrangeData("nov", os.getcwd(), nov_data_det)

In [ ]:
downloadDataUsingThread(dec_data_det)
dec_data_dir = arrangeData("dec", os.getcwd(), dec_data_det)

In [ ]:
downloadDataUsingThread(jan_data_det)
jan_data_dir = arrangeData("jan", os.getcwd(), jan_data_det)

In [ ]:
downloadDataUsingThread(feb_data_det)
feb_data_dir = arrangeData("feb", os.getcwd(), feb_data_det)

In [ ]:
downloadDataUsingThread(mar_data_det)
mar_data_dir = arrangeData("mar", os.getcwd(), mar_data_det)

In [ ]:
extractFiles(nov_data_dir)
extractFiles(dec_data_dir)
extractFiles(jan_data_dir)
extractFiles(feb_data_dir)
extractFiles(mar_data_dir)

In [ ]:
nov_data_dir = getDataDirectory("nov")
extractFiles(nov_data_dir)
cleanup(nov_data_dir)

In [ ]:
dec_data_dir = getDataDirectory("dec")
extractFiles(dec_data_dir)
cleanup(dec_data_dir)

In [ ]:
jan_data_dir = getDataDirectory("jan")
extractFiles(jan_data_dir)
cleanup(jan_data_dir)

In [ ]:
feb_data_dir = getDataDirectory("feb")
extractFiles(feb_data_dir)
cleanup(feb_data_dir)

In [ ]:
mar_data_dir = getDataDirectory("mar")
extractFiles(mar_data_dir)
cleanup(mar_data_dir)

In [ ]:
#stack, mosaic and clip
